In [ ]:

# Deep Convolutional GANs
encoded_image=[]
# Importing the libraries
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt
import random
import warnings

warnings.filterwarnings("ignore")
!rm -rf "sample_data"

In [ ]:
ngpu=4
import os
number_of_persons=1
encodedir='./encodedimage/'
if not os.path.exists(encodedir):
    os.makedirs(encodedir)
    print("Directory " , encodedir ,  " Created ")
else:    
    print("Directory " , encodedir ,  " already exists")
for n_person_directory in range(1,number_of_persons+1):
  dirName1 = './images/'+'test'+str(n_person_directory)+'/person'+str(n_person_directory)+'/'

  # Create target Directory if don't exist
  if not os.path.exists(dirName1):
      os.makedirs(dirName1)
      print("Directory " , dirName1 ,  " Created ")
  else:    
      print("Directory " , dirName1 ,  " already exists")


  dirName='./results_of_person'+str(n_person_directory)+'/'
  # Create target Directory if don't exist
  if not os.path.exists(dirName):
      os.makedirs(dirName)
      print("Directory " , dirName ,  " Created ")
  else:    
      print("Directory " , dirName ,  " already exists")


In [ ]:
for n_person_loop in range(1,number_of_persons+1):
  random.seed(n_person_loop)
  torch.manual_seed(n_person_loop)
  dirName='./results_of_person'+str(n_person_loop)+'/'
  # Setting some hyperparameters
  batchSize = 1 # We set the size of the batch. batchSize=1 means we will provide only one image of one person
  imageSize = 64 #It is actually not the size, I have multiplied it with the factor of 8 to make it 512x512
  
  # Creating the transformations
  TRANSFORM_IMG = transforms.Compose([
      transforms.Resize(imageSize*32),
      transforms.CenterCrop(imageSize*32),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.5, 0.5, 0.5],
                          std=[0.5, 0.5, 0.5] )
      ])
  TRAIN_DATA_PATH = './images/test'+str(n_person_loop)+'/'


  # Loading the dataset
  dataset = dset.ImageFolder(root=TRAIN_DATA_PATH, transform=TRANSFORM_IMG) # Applying transformation on each image.
  dataloader = torch.utils.data.DataLoader(dataset, batch_size = batchSize, shuffle = False, num_workers = 0) # We use dataLoader to get the images of the training set batch by batch.
  

  # Decide which device we want to run on
  device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

  # Plot some training images
  real_batch = next(iter(dataloader))
  plt.figure(figsize=(32,32))
  plt.axis("off")
  plt.title("Training Images")
  plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:imageSize*32], padding=2, normalize=True).cpu(),(1,2,0)))


In [ ]:



for n_person_loop in range(1,number_of_persons+1):
  random.seed(n_person_loop)
  torch.manual_seed(n_person_loop)
  dirName='./results_of_person'+str(n_person_loop)+'/'
  # Setting some hyperparameters
  batchSize = 1 # We set the size of the batch. batchSize=1 means we will provide only one image of one person
  imageSize =64 #It is actually not the size, I have multiplied it with the factor of 8 to make it 512x512
  
  # Creating the transformations
  TRANSFORM_IMG = transforms.Compose([
      transforms.Resize(imageSize*32),
      transforms.CenterCrop(imageSize*32),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.5, 0.5, 0.5],
                          std=[0.5, 0.5, 0.5] )
      ])
  TRAIN_DATA_PATH = './images/test'+str(n_person_loop)+'/'


  # Loading the dataset
  dataset = dset.ImageFolder(root=TRAIN_DATA_PATH, transform=TRANSFORM_IMG) # Applying transformation on each image.
  dataloader = torch.utils.data.DataLoader(dataset, batch_size = batchSize, shuffle = False, num_workers = 0) # We use dataLoader to get the images of the training set batch by batch.
  

  # Decide which device we want to run on
  device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
  # device = torch.device("cpu") #If you strictly want to use the cpu
  

  # Plot some training images
  real_batch = next(iter(dataloader))
  plt.figure(figsize=(32,32))
  plt.axis("off")
  plt.title("Training Images")
  plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:imageSize*32], padding=2, normalize=True).cpu(),(1,2,0)))


  def weights_init(m):
      classname = m.__class__.__name__
      if classname.find('Conv') != -1:
          m.weight.data.normal_(0.0, 0.02)
      elif classname.find('BatchNorm') != -1:
          m.weight.data.normal_(1.0, 0.02)
          m.bias.data.fill_(0)


  # Defining the generator

  class G(nn.Module): # We introduce a class to define the generator.

      def __init__(self,ngpu): # We introduce the __init__() function that will define the architecture of the generator.
          super(G, self).__init__() # We inherit from the nn.Module tools.
          self.ngpu=ngpu
          
          self.main = nn.Sequential( # We create a meta module of a neural network that will contain a sequence of modules (convolutions, full connections, etc.).
              nn.ConvTranspose2d(100, imageSize*256, 4, 1, 0, bias = False), # We start with an inversed convolution.
              nn.BatchNorm2d(imageSize*256), # We normalize all the features along the dimension of the batch.
              nn.ReLU(True), # We apply a ReLU rectification to break the linearity.              

              nn.ConvTranspose2d(imageSize*256, imageSize*128, 4, 2, 1, bias = False), # We start with an inversed convolution.
              nn.BatchNorm2d(imageSize*128), # We normalize all the features along the dimension of the batch.
              nn.ReLU(True), # We apply a ReLU rectification to break the linearity.

              nn.ConvTranspose2d(imageSize*128, imageSize*64, 4, 2, 1, bias = False), # We start with an inversed convolution.
              nn.BatchNorm2d(imageSize*64), # We normalize all the features along the dimension of the batch.
              nn.ReLU(True), # We apply a ReLU rectification to break the linearity.

              nn.ConvTranspose2d(imageSize*64, imageSize*32, 4, 2, 1, bias = False), # We start with an inversed convolution.
              nn.BatchNorm2d(imageSize*32), # We normalize all the features along the dimension of the batch.
              nn.ReLU(True), # We apply a ReLU rectification to break the linearity.

              nn.ConvTranspose2d(imageSize*32, imageSize*16, 4, 2, 1, bias = False), # We start with an inversed convolution.
              nn.BatchNorm2d(imageSize*16), # We normalize all the features along the dimension of the batch.
              nn.ReLU(True), # We apply a ReLU rectification to break the linearity.
              nn.ConvTranspose2d(imageSize*16, imageSize*8, 4, 2, 1, bias = False), # We add another inversed convolution.
              nn.BatchNorm2d(imageSize*8), # We normalize again.
              nn.ReLU(True), # We apply another ReLU.
              nn.ConvTranspose2d(imageSize*8, imageSize*4, 4, 2, 1, bias = False), # We add another inversed convolution.
              nn.BatchNorm2d(imageSize*4), # We normalize again.
              nn.ReLU(True), # We apply another ReLU.
              nn.ConvTranspose2d(imageSize*4, imageSize*2, 4, 2, 1, bias = False), # We add another inversed convolution.
              nn.BatchNorm2d(imageSize*2), # We normalize again.
              nn.ReLU(True), # We apply another ReLU.
              nn.ConvTranspose2d(imageSize*2,imageSize, 4, 2, 1, bias = False), # We add another inversed convolution.
              nn.BatchNorm2d(imageSize), # We normalize again.
              nn.ReLU(True), # We apply another ReLU.
              nn.ConvTranspose2d(imageSize, 3, 4, 2, 1, bias = False), # We add another inversed convolution.
              nn.Tanh() # We apply a Tanh rectification to break the linearity and stay between -1 and +1.
              
              )
      
      def forward(self, input): # We define the forward function that takes as argument an input that will be fed to the neural network, and that will return the output containing the generated images.
          output = self.main(input) # We forward propagate the signal through the whole neural network of the generator defined by self.main.
          global encoded_image
          encoded_image=torch.atanh(output)
          
          return output # We return the output containing the generated images.
      

  
  # Create the generator
  netG = G(ngpu).to(device)

  # Handle multi-gpu if desired
  if (device.type == 'cuda') and (ngpu > 1):
      netG = nn.DataParallel(netG, list(range(ngpu)))
  # Apply the weights_init function to randomly initialize all weights
  #  to mean=0, stdev=0.2.
  netG.apply(weights_init)

  # Print the model
  #print(netG)

  # Defining the discriminator

  class D(nn.Module): # We introduce a class to define the discriminator.

      def __init__(self,ngpu): # We introduce the __init__() function that will define the architecture of the discriminator.
          super(D, self).__init__() # We inherit from the nn.Module tools.
          self.ngpu=ngpu
          self.main = nn.Sequential( # We create a meta module of a neural network that will contain a sequence of modules (convolutions, full connections, etc.).
              nn.Conv2d(3, imageSize, 4, 2, 1, bias = False), # We start with a convolution.
              nn.LeakyReLU(0.2, inplace = True), # We apply a LeakyReLU.
              nn.Conv2d(imageSize, imageSize*2, 4, 2, 1, bias = False), # We add another convolution.
              nn.BatchNorm2d(imageSize*2), # We normalize all the features along the dimension of the batch.
              nn.LeakyReLU(0.2, inplace = True), # We apply another LeakyReLU.
              nn.Conv2d(imageSize*2, imageSize*4, 4, 2, 1, bias = False), # We add another convolution.
              nn.BatchNorm2d(imageSize*4), # We normalize again.
              nn.LeakyReLU(0.2, inplace = True), # We apply another LeakyReLU.
              nn.Conv2d(imageSize*4, imageSize*8, 4, 2, 1, bias = False), # We add another convolution.
              nn.BatchNorm2d(imageSize*8), # We normalize again.
              nn.LeakyReLU(0.2, inplace = True), # We apply another LeakyReLU.

              nn.Conv2d(imageSize*8, imageSize*16, 4, 2, 1, bias = False), # We add another convolution.
              nn.BatchNorm2d(imageSize*16), # We normalize again.  #16
              nn.LeakyReLU(0.2, inplace = True), # We apply another LeakyReLU.
              nn.Conv2d(imageSize*16, imageSize*32, 4, 2, 1, bias = False), # We add another convolution.
              nn.BatchNorm2d(imageSize*32), # We normalize again.  #16
              nn.LeakyReLU(0.2, inplace = True), # We apply another LeakyReLU.  

              nn.Conv2d(imageSize*32, imageSize*64, 4, 2, 1, bias = False), # We add another convolution.
              nn.BatchNorm2d(imageSize*64), # We normalize again.  #16
              nn.LeakyReLU(0.2, inplace = True), # We apply another LeakyReLU.                 

              nn.Conv2d(imageSize*64, imageSize*128, 4, 2, 1, bias = False), # We add another convolution.
              nn.BatchNorm2d(imageSize*128), # We normalize again.  #16
              nn.LeakyReLU(0.2, inplace = True), # We apply another LeakyReLU.  



              nn.Conv2d(imageSize*128, imageSize*256, 4, 2, 1, bias = False), # We add another convolution.
              nn.BatchNorm2d(imageSize*256), # We normalize again.  #16
              nn.LeakyReLU(0.2, inplace = True), # We apply another LeakyReLU.  


              nn.Conv2d(imageSize*256, 1, 4, 1, 0, bias = False), # We add another convolution.  #16
              nn.Sigmoid() # We apply a Sigmoid rectification to break the linearity and stay between 0 and 1.
              
              )

      def forward(self, input): # We define the forward function that takes as argument an input that will be fed to the neural network, and that will return the output which will be a value between 0 and 1.
          output = self.main(input) # We forward propagate the signal through the whole neural network of the discriminator defined by self.main.
          return output.view(-1) # We return the output which will be a value between 0 and 1.

  # Create the Discriminator
  netD = D(ngpu).to(device)
  if (device.type == 'cuda') and (ngpu > 1):
      netD = nn.DataParallel(netD, list(range(ngpu)))

  # Apply the weights_init function to randomly initialize all weights
  #  to mean=0, stdev=0.2.
  netD.apply(weights_init)

  # Print the model
  #print(netD)

  # Training the DCGANs

  criterion = nn.BCELoss() # We create a criterion object that will measure the error between the prediction and the target.
  optimizerD = optim.Adam(netD.parameters(), lr = 0.0002, betas = (0.5, 0.999)) # We create the optimizer object of the discriminator.
  optimizerG = optim.Adam(netG.parameters(), lr = 0.0002, betas = (0.5, 0.999)) # We create the optimizer object of the generator.
  
  G_losses=[]
  D_losses=[]
  n_epochs=1
  for epoch in range(1,n_epochs+1): # We iterate over number of epochs.

      for i, data in enumerate(dataloader, 0): # We iterate over the images of the dataset.
          
          # 1st Step: Updating the weights of the neural network of the discriminator

          netD.zero_grad() # We initialize to 0 the gradients of the discriminator with respect to the weights.
          
          # Training the discriminator with a real image of the dataset
          real = data[0].to(device) # We get a real image of the dataset which will be used to train the discriminator.
          input = Variable(real) # We wrap it in a variable.
          target = Variable(torch.ones(input.size()[0]).to(device)) # We get the target.
          output = netD(input) # We forward propagate this real image into the neural network of the discriminator to get the prediction (a value between 0 and 1).
          errD_real = criterion(output, target) # We compute the loss between the predictions (output) and the target (equal to 1).
          
          # Training the discriminator with a fake image generated by the generator
          noise = Variable(torch.randn(input.size()[0], 100, 1, 1).to(device)) # We make a random input vector (noise) of the generator.
          fake = netG(noise) # We forward propagate this random input vector into the neural network of the generator to get some fake generated images.
          target = Variable(torch.zeros(input.size()[0]).to(device)) # We get the target.
          output = netD(fake.detach()) # We forward propagate the fake generated images into the neural network of the discriminator to get the prediction (a value between 0 and 1).
          errD_fake = criterion(output, target) # We compute the loss between the prediction (output) and the target (equal to 0).

          # Backpropagating the total error
          errD = errD_real + errD_fake # We compute the total error of the discriminator.
          errD.backward() # We backpropagate the loss error by computing the gradients of the total error with respect to the weights of the discriminator.
          optimizerD.step() # We apply the optimizer to update the weights according to how much they are responsible for the loss error of the discriminator.

          # 2nd Step: Updating the weights of the neural network of the generator

          netG.zero_grad() # We initialize to 0 the gradients of the generator with respect to the weights.
          target = Variable(torch.ones(input.size()[0]).to(device)) # We get the target.
          output = netD(fake) # We forward propagate the fake generated images into the neural network of the discriminator to get the prediction (a value between 0 and 1).
          errG = criterion(output, target) # We compute the loss between the prediction (output between 0 and 1) and the target (equal to 1).
          errG.backward() # We backpropagate the loss error by computing the gradients of the total error with respect to the weights of the generator.
          optimizerG.step() # We apply the optimizer to update the weights according to how much they are responsible for the loss error of the generator.
          
          # 3rd Step: Printing the losses and saving the real images and the generated images of the minibatch every 100 steps

          #print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f' % (epoch, n_epochs, i, len(dataloader), errD.data, errG.data)) # We print les losses of the discriminator (Loss_D) and the generator (Loss_G).
          G_losses.append(errG.item())
          D_losses.append(errD.item())
          vutils.save_image(real, '%s/real_samples.png' % dirName, normalize = True) # We save the real images of the minibatch.
          fake = netG(noise) # We get our fake generated images.
          vutils.save_image(fake.data, '%s/fake_samples_epoch_%04d.png' % (dirName, epoch), normalize = True) # We also save the fake generated images of the minibatch.

  plt.figure(figsize=(10,5))
  plt.title("Generator and Discriminator Loss During Training")
  plt.plot(G_losses,label="G")
  plt.plot(D_losses,label="D")
  plt.xlabel("iterations")
  plt.ylabel("Loss")
  plt.legend()
  plt.show()          
  dir2="saved_models/fake_images"
  dir3="saved_models/fake_images/fake_images_person"
  if not os.path.exists(dir2):
      os.makedirs(dir2)
      print("Directory " , dir2 ,  " Created ")
  else:    
      print("Directory " , dir2 ,  " already exists")

  dir1="models/pytorch"
  if not os.path.exists(dir1):
      os.makedirs(dir1)
      print("Directory " , dir1 ,  " Created ")
  else:    
      print("Directory " , dir1 ,  " already exists")
  
  pathG="models/pytorch/G.pth_person"+str(n_person_loop)
  
  torch.save(netG, pathG)

  pathG_weights="models/pytorch/G.h5_person"+str(n_person_loop)

  torch.save(netG.state_dict(),pathG_weights )
  


In [ ]:

vutils.save_image(encoded_image, '%s/encoded.png' % encodedir, normalize = True)


In [ ]:

%pylab inline
print ("\n\n\nEncoded image below")
imge=mpimg.imread("./encodedimage/encoded.png")
imgplote = plt.imshow(imge)
plt.show()


In [ ]:
for n_person_test in range(1,number_of_persons+1):
  batchSize=1
  random.seed(n_person_loop)
  torch.manual_seed(n_person_test)
  dir3_test="saved_models/fake_images/fake_images_person"
  pathG_test="models/pytorch/G.pth_person"+str(n_person_test)
  

  netGG = torch.load(pathG_test)
  netGG.eval()
  
  noi = Variable(torch.randn(batchSize, 100, 1, 1).to(device)) # We make a random input vector (noise) of the generator.
  fa = netGG(noi)

  vutils.save_image(fa.detach(), dir3_test+str(n_person_test)+".png", normalize=True)


In [ ]:
n_persons=1
for n_person_test_weight in range(1,n_persons+1):
  random.seed(n_person_test_weight)
  torch.manual_seed(n_person_test_weight)
  dir4="saved_models/fake_images_by_weights"
  dir5="saved_models/fake_images_by_weights/fake_images_person"
  if not os.path.exists(dir4):
      os.makedirs(dir4)
      print("Directory " , dir4 ,  " Created ")
  else:    
      print("Directory " , dir4 ,  " already exists")

  pathG_weights="models/pytorch/G.h5_person"+str(n_person_test_weight)
  
  netG.load_state_dict(torch.load(pathG_weights))
  netG.eval()
 



  noiw = Variable(torch.randn(16, 100, 1, 1).to(device)) # We make a random input vector (noise) of the generator.
  faw= netG(noi)

  vutils.save_image(fa.detach(), dir5+str(n_person_test_weight)+".png", normalize=True)

In [ ]:
# !rm -rf "saved_models"
# !rm -rf "models"
